In [6]:
import numpy as np
import graph_tool.all as gt

iterations = 1000  # number of page rank iterations

size = 200
graph_er = np.zeros((size, size))
p = 0.05

In [7]:
# Erdösz-Renyi
for i in range(size):
    for j in range(i):
        rand = np.random.rand()
        if rand <= p:
            graph_er[i, j] = 1
            graph_er[j, i] = 1

# Calculate page rank
row_sum = np.sum(graph_er, axis=1)
row_sum[row_sum == 0] = 1
B = graph_er / row_sum
B_l = B.copy()
r = np.ones((size, 1)) / size
for i in range(iterations - 1):
    B_l = np.matmul(B, B_l)
page_rank_er = np.matmul(B_l, r)

In [8]:
# Barabasi Albert
graph_ba = np.zeros((size, size))
m = 5
graph_ba[0:m, 0:m] = 1  # Start with a small fully connected graph

for i in range(m, size):  # Generate the rest of the graph
    for _ in range(m):
        rand = np.random.rand()
        cum_prob = 0
        for j in range(i):
            cum_prob += np.sum(graph_ba[j]) / np.sum(graph_ba)
            if rand <= cum_prob:
                graph_ba[i, j] = 1
                graph_ba[j, i] = 1
                break

# Calculate page rank
row_sum = np.sum(graph_ba, axis=1)
row_sum[row_sum == 0] = 1
B = graph_ba / row_sum
r = np.ones((size, 1)) / size
B_l = B.copy()
for i in range(iterations - 1):
    B = np.matmul(B, B_l)
B = np.matmul(B, r)
page_rank_ba = B.copy()

In [9]:
import pandas as pd

# Show page ranks
print(pd.DataFrame(page_rank_er, columns=["Page rank Erdösz-Renyi"]))
print(pd.DataFrame(page_rank_ba, columns=["Page rank Barabasi-Albert"]))

     Page rank Erdösz-Renyi
0                  0.003448
1                  0.003448
2                  0.004433
3                  0.004433
4                  0.003448
..                      ...
195                0.004926
196                0.004433
197                0.004433
198                0.006404
199                0.006404

[200 rows x 1 columns]
     Page rank Barabasi-Albert
0                     0.019546
1                     0.015320
2                     0.013735
3                     0.029054
4                     0.021659
..                         ...
195                   0.002641
196                   0.002641
197                   0.002641
198                   0.002641
199                   0.002641

[200 rows x 1 columns]


In [10]:
# Plot results
g = gt.Graph(directed=False)
g.add_vertex(size)
for i in range(size):
    for j in range(i):
        if graph_er[i, j] == 1:
            g.add_edge(g.vertex(i), g.vertex(j))

pos = gt.sfdp_layout(g)
gt.graph_draw(
    g,
    pos=pos,
    vertex_fill_color=page_rank_er,
    vertex_size=10,
    output_size=(1000, 1000),
    output="erdos_renyi.png",
)

g = gt.Graph(directed=False)
g.add_vertex(size)
for i in range(size):
    for j in range(i):
        if graph_ba[i, j] == 1:
            g.add_edge(g.vertex(i), g.vertex(j))

pos = gt.sfdp_layout(g)
gt.graph_draw(
    g,
    pos=pos,
    vertex_fill_color=page_rank_ba,
    vertex_size=10,
    output_size=(1000, 1000),
    output="barabasi_albert.png",
)

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7f1f3120b910, at 0x7f1f30e93f10>

![Erdösz-Renyi](erdos_renyi.png)
![Barabasi Albert](barabasi_albert.png)